# NOTEBOOK PARA OBTENER LOS HITS Y MEJORES HITS
Esto incluye la limpieza que se requiere ya que tienen dobles espacios en cada línea.

In [1]:
#Primero algunas variables importantes de ubicación de los archivos y librerías
import os
import pandas as pd
import re
import math

extractpath = "U:\MASTER INT. ARTIFICIAL\TFM_DATA\TXTs" #Este path es para mi laptop en casa
#extractpath = "F:\\DATA_TFM\\TXTs" #Este path es para mi PC en el trabajo
#extractpath = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\TXTs" #Este es para mi no tan nueva PC-Desktop
#csv_path = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\\" #Path para mi PC Desktop
csv_path = "U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\\" #Path para mi latptop en casa
#csv_path = "F:\\DATA_TFM\\CSVs\\"

txtfilesarr = os.listdir(extractpath); txtfilesarr = sorted(txtfilesarr)
csvarr = os.listdir(csv_path); csvarr = sorted(csvarr)

In [2]:
# Funcion para obtener los indices de un archivo
def indices(df):
    ind_startev = [];ind_runid = [];ind_weights = [];ind_muon = []
    ind_bbfit = [];ind_bb_selp = [];ind_endev = [];#ind_aafit = []
    for i in range(data.shape[0]):
        tempstr = data[0][i]
        if "start_event" in tempstr: ind_startev.append(i)
        if "UTC" in tempstr: ind_runid.append(i)
        if "weights" in tempstr: ind_weights.append(i)
        if "muon" in tempstr: ind_muon.append(i)
        #if "aafit" in tempstr:  ##NO LO CALCULARE!!! OBTENDRE LOS VALORES DE OTRA FORMA
        #    ind_aafit.append(i) ##COMO TENGO EL INDICE DE BBFIT, RESTO 1 LINEA Y ASIGNO VALORES
        if "bbfit" in tempstr:   ##SEGUN LA EXISTENCIA O NO DE LA LINEA!!!! EUREKA!!!
            ind_bbfit.append(i)
        if "selected" in tempstr: ind_bb_selp.append(i)
        if "end_event" in tempstr: ind_endev.append(i)
    return(ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev)

#Función para limpiar un hit (lista) de elementos tipo dobles espacios y simples
def limpiahit(templist):
    cleanlist = []
    for j in range(len(templist)):
        if templist[j]!="":
            cleanlist.append(templist[j])
    return(cleanlist)

#Función para verificar si los indices tienen el mismo # de elementos
def longitudes(i1,i2,i3,i4,i5,i6,i7):
    a=len(i1);b=len(i2);c=len(i3);d=len(i4);e=len(i5);f=len(i6);g=len(i7)
    #print(a,b,c,d,e,f,g)
    if (a==b and b==c and c==d and d==e and e==f and f==g): return(True)
    else: return(False)
    return()

In [3]:
file = csv_path+csvarr[0]
print("El archivo trabajando es: ",file)
data = pd.read_csv(file, header=None)

print(data[0][0:8])
data[0][130:140]

El archivo trabajando es:  U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025800_anumu_CC_a_reco.i3.csv
0                                       start_event  1
1    25800 73557 0 1170788183 2007-02-06 18:56:23.0...
2              weights  91530000.0 3191000.0 1000000.0
3    muon -0.0718400384771 -0.325562174369 0.942787...
4    aafit  -0.294267285203 -0.300496709702 0.90725...
5    bbfit_track  nan nan 0.953328222068 0.09406368...
6    hit    1  3   2  1  -9.782512  37.711367 -156....
7    hit    2  3   2  2 -10.540966  37.058107 -156....
Name: 0, dtype: object


130    hit  125  1  22  2   4.278801  96.848412  125....
131    hit  126  1   7  2   4.457698  96.844844  -92....
132                               BBFit selected pulses:
133    hit  1  4  19  0  45.532615  9.219245   90.040...
134    hit  2  4  20  0  45.484239  9.361442  104.567...
135    hit  3  4  21  0  44.788333  9.655823  119.094...
136    hit  4  4  22  0  45.134250  8.552834  133.620...
137    hit  5  4  23  0  45.124893  8.550070  148.146...
138                                           end_event 
139                                       start_event  2
Name: 0, dtype: object

In [4]:
###################################
### OBTENIENDO LOS MEJORES HITS ###
###################################

## Por el momento será para un solo archivo, posteriormente haré el bucle.

#Leyendo el archivo csv ya sin las líneas no deseadas en un dataframe
file = csv_path+csvarr[0]
print("El archivo trabajando es: ",file)
data = pd.read_csv(file, header=None)

#Obteniendo los indices necesarios
ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev = indices(data)

#Necesito encontrar las líneas entre bbfit_track y "selected pulses" y entre selected pulses y end_event
print(ind_bbfit[0], ind_bb_selp[0], ind_endev[0])

El archivo trabajando es:  U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025800_anumu_CC_a_reco.i3.csv
5 132 138


In [5]:
difrawhits = ind_bb_selp[0] - ind_bbfit[0]
difselhits = ind_endev[0] - ind_bb_selp[0]

start_rawhit = ind_bbfit[0] + 1; #end_rawhit = ind_bb_selp
start_selhit = ind_bb_selp[0]+1

raw_hits = []; hits_raw_limpios = []
sel_hits = []; sel_hits_limpios = []

#Obteniendo solo los raw_hits de un evento
for i in range(difrawhits-1):
    temprawhit = list(data[0][start_rawhit+i].split(" "))
    raw_hits.append(temprawhit)
#Limpiamos los raw hits obtenidos del evento
for i in range(len(raw_hits)):
    hits_raw_limpios.append(limpiahit(raw_hits[i]))

#Obteniendo solo los selected pulses (hits) del mismo evento
for i in range(difselhits-1):
    tempselhit = list(data[0][start_selhit+i].split(" "))
    sel_hits.append(tempselhit)
#Limpiamos los raw hits obtenidos del evento
for i in range(len(sel_hits)):
    sel_hits_limpios.append(limpiahit(sel_hits[i]))

In [6]:
print("RAW HITS LIMPIOS")
print(len(raw_hits),hits_raw_limpios)
print("SELECTED HITS LIMPIOS")
print(len(sel_hits_limpios),sel_hits_limpios)

RAW HITS LIMPIOS
126 [['hit', '1', '3', '2', '1', '-9.782512', '37.711367', '-156.955892', '0.233261', '0.667525', '-0.707106', '124.401378', '1.230586', '51.116943'], ['hit', '2', '3', '2', '2', '-10.540966', '37.058107', '-156.955892', '-0.694724', '-0.131753', '-0.707106', '2265.336063', '1.445710', '72.250366'], ['hit', '3', '4', '17', '1', '44.808764', '8.549579', '60.984108', '-0.192247', '-0.680472', '-0.707106', '1387.373901', '1.239539', '63.667297'], ['hit', '4', '4', '22', '0', '45.134250', '8.552834', '133.620108', '0.205998', '-0.676436', '-0.707106', '251.590850', '6.414619', '314.025879'], ['hit', '5', '4', '22', '1', '45.360462', '9.528035', '133.620108', '0.482812', '0.516617', '-0.707106', '1115.562612', '0.990252', '206.813812'], ['hit', '6', '4', '22', '1', '45.360462', '9.528035', '133.620108', '0.482812', '0.516617', '-0.707106', '416.983206', '1.442759', '206.813812'], ['hit', '7', '4', '22', '2', '44.402881', '9.236419', '133.620108', '-0.688809', '0.159819', '-

In [27]:
#Código para analizar la correlación entre selected pulses y raw hits y asignar el módulo óptico.

#Idea... pasemos a dataframe las listas que tenemos

columnas = ['hit','#hit','lin_det','piso_det','mód_opt',
            'X\'','Y\'','Z\'','X','Y','Z','tiempo','amp','frec_hits']

raw_hits_ev_df = pd.DataFrame(hits_raw_limpios,columns=columnas)
sel_hits_ev_df = pd.DataFrame(sel_hits_limpios,columns=columnas)

#print(raw_hits_ev_df.head(5))
#print(sel_hits_ev_df.head(5))

#Tomo como base el primer elemento de los selected pulses
#print(sel_hits_limpios[0])

#Debo comparar contra todos y cada uno de los raw hits
#Sin embargo esto es fuerza bruta y no se si exista alguna mejor forma... como buscar por pisos primero


In [78]:
#raw_hits_ev_df.sample(10)
sel_hits_ev_df

,hit,#hit,lin_det,piso_det,mód_opt,X',Y',Z',X,Y,Z,tiempo,amp,frec_hits
0,hit,1,4,19,0,45.532615,9.219245,90.040108,0.693344,0.138831,-0.707106,137.146191,3.152491,327.587128
1,hit,2,4,20,0,45.484239,9.361442,104.567108,0.634163,0.312791,-0.707106,167.488297,8.169946,82.550049
2,hit,3,4,21,0,44.788333,9.655823,119.094108,-0.217186,0.672927,-0.707106,201.236802,24.179034,60.100555
3,hit,4,4,22,0,45.134250,8.552834,133.620108,0.205998,-0.676436,-0.707106,251.590850,11.378742,314.025879
4,hit,5,4,23,0,45.124893,8.550070,148.146108,0.194551,-0.679817,-0.707106,315.268888,3.454047,73.604584


In [58]:
#Posiciones a comparar 8, 9 y 10 del df

print(sel_hits_ev_df.iloc[0])
mod_optico=sel_hits_ev_df.iloc[0][3]
mod_optico

raw_hits_ev_df.iloc[76]

#if sel_hits_ev_df.iloc[78]

#raw_hits_ev_df
#sel_hits_ev_df[0]

hit                 hit
#hit                  1
lin_det               4
piso_det             19
mód_opt               0
X'            45.532615
Y'             9.219245
Z'            90.040108
X              0.693344
Y              0.138831
Z             -0.707106
tiempo       137.146191
amp            3.152491
frec_hits    327.587128
Name: 0, dtype: object


hit                 hit
#hit                 77
lin_det               4
piso_det             19
mód_opt               0
X'            45.532615
Y'             9.219245
Z'            90.040108
X              0.693344
Y              0.138831
Z             -0.707106
tiempo       137.146191
amp            0.879543
frec_hits    327.587128
Name: 76, dtype: object

In [93]:
lindet_search = sel_hits_ev_df.iloc[0][2]
storey_search = sel_hits_ev_df.iloc[0][3]

print(lindet_search,storey_search)

4 19


In [98]:
#### FILTRO PARA BUSCAR EL MODULO OPTICO QUE CORRESPONDE AL SELECTED HIT

lindet_search = sel_hits_ev_df.iloc[0][2] #Linea detector a buscar
storey_search = sel_hits_ev_df.iloc[0][3] #Piso detector a buscar

#Primero busqué por piso detector (me pareció que era más corto ya que son menos lineas
tempdf = raw_hits_ev_df.loc[raw_hits_ev_df['piso_det'].isin([storey_search])]
#Posteriormente filtré por línea del detector
tempdf = tempdf.loc[tempdf['lin_det'].isin([lindet_search])]

print(sel_hits_ev_df.iloc[0])
tempdf

hit                 hit
#hit                  1
lin_det               4
piso_det             19
mód_opt               0
X'            45.532615
Y'             9.219245
Z'            90.040108
X              0.693344
Y              0.138831
Z             -0.707106
tiempo       137.146191
amp            3.152491
frec_hits    327.587128
Name: 0, dtype: object


,hit,#hit,lin_det,piso_det,mód_opt,X',Y',Z',X,Y,Z,tiempo,amp,frec_hits
75,hit,76,4,19,0,45.532615,9.219245,90.040108,0.693344,0.138831,-0.707106,296.559100,1.089169,327.587128
76,hit,77,4,19,0,45.532615,9.219245,90.040108,0.693344,0.138831,-0.707106,137.146191,0.879543,327.587128
77,hit,78,4,19,1,44.584223,9.539780,90.040108,-0.466903,0.531039,-0.707106,153.189412,0.772948,281.085968


## Ahora el código para TODOS los eventos en un mismo archivo

In [ ]:
#Igual y se requiere un dataframe para guardar la data
hits = pd.DataFrame()

#Leyendo el archivo csv ya sin las líneas no deseadas en un dataframe
file = csv_path+csvarr[0]
print("El archivo trabajando es: ",file)
data = pd.read_csv(file, header=None)
#Obteniendo los indices necesarios
ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev = indices(data)

for z in range (len(ind_startev)):
    #Definición de variables auxiliares
    difrawhits = ind_bb_selp[0] - ind_bbfit[0]; difselhits = ind_endev[0] - ind_bb_selp[0]
    start_rawhit = ind_bbfit[0] + 1; start_selhit = ind_bb_selp[0]+1
    raw_hits = []; hits_raw_limpios = []; sel_hits = []; sel_hits_limpios = []
    #Obteniendo solo los raw_hits de un evento
    for i in range(difrawhits-1):
        temprawhit = list(data[0][start_rawhit+i].split(" "))
        raw_hits.append(temprawhit)
    #Limpiamos los raw hits obtenidos del evento
    for i in range(len(raw_hits)):
        hits_raw_limpios.append(limpiahit(raw_hits[i]))
    #Obteniendo solo los selected pulses (hits) del mismo evento
    for i in range(difselhits-1):
        tempselhit = list(data[0][start_selhit+i].split(" "))
        sel_hits.append(tempselhit)
    #Limpiamos los raw hits obtenidos del evento
    for i in range(len(sel_hits)):
        sel_hits_limpios.append(limpiahit(sel_hits[i]))
    #Debemos añadir la data a un solo dataframe o lista

In [ ]:

for z in range (len(ind_startev)):
    print (z)
